In [1]:
!pip install datasets

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

In [7]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

 Load the model and tokenizer

In [3]:
model_name = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def get_response(messages):

    inputs = tokenizer.encode(user_prompt, return_tensors="pt")

    outputs = model.generate(inputs, max_length=300, num_return_sequences=1)
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_answer



### Load the dataset

In [5]:
dataset=load_dataset("bigbio/med_qa")

In [7]:
dataset["train"][:5]


{'meta_info': ['step2&3', 'step2&3', 'step1', 'step1', 'step1'],
 'question': ['A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient?',
  'A 3-month-old baby died suddenly at night while asleep. His mother noticed that he had died only after she awoke in the morning. No cause of death was determined based on the autopsy. Which of the following precautions could have prevented the death of the baby?',
  "A mother brings her 3-week-old infant to the pediatrician's off

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1273
    })
    validation: Dataset({
        features: ['meta_info', 'question', 'answer_idx', 'answer', 'options'],
        num_rows: 1272
    })
})

In [9]:
first_question=''' Which of the following is a risk factor for heart disease?
A) High blood pressure
B) Low cholesterol
C) Low body mass index
D) Moderate exercise
'''

In [10]:
first_question_answer='(A) High blood pressure'

In [11]:
second_question=''' What is the primary cause of type 1 diabetes?
A) Excess sugar intake
B) Autoimmune destruction of insulin-producing cells
C) Lack of physical exercise
D) High fat diet
'''

In [12]:
second_question_answer='B) Autoimmune destruction of insulin-producing cells'

In [13]:
third_question='''Which vitamin is primarily responsible for calcium absorption in the body?
A) Vitamin C
B) Vitamin A
C) Vitamin D
D) Vitamin K
'''

In [14]:
third_question_answer='C) Vitamin D'

In [15]:
fourth_question='''What is the first-line treatment for hypertension in a patient with chronic kidney disease?
A) Beta-blockers
B) Calcium channel blockers
C) ACE inhibitors
D) Diuretics
'''


## Define the System Role

In [16]:
system_role='You are a helpful assistant that answers multiple choice  questions about medical knowledge'

##   Few-Shot Prompting

In [17]:
user_prompt=f'''Question:{first_question}
Answer:{first_question_answer}

Question:{second_question}
Answer:{second_question_answer}

Question:{third_question}
Answer:{third_question_answer}

Question:{fourth_question}
Answer:
'''

In [18]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
         ]

In [19]:
print(f'The answer is :{get_response(messages)}')

The answer is :C



##                One Shot Prompting

In [20]:
user_prompt=f'''Question:{first_question}
Answer:{first_question_answer}

Question:{fourth_question}
Answer:
'''

In [21]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
         ]

In [22]:
print(f'The answer is :{get_response(messages)}')


The answer is :C


## Zero Shot Prompting

In [23]:
user_prompt=f'''Question:{fourth_question}
Answer:
'''

In [24]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
         ]

In [25]:
print(f'The answer is :{get_response(messages)}')

The answer is :C
